In [34]:
from Features import *
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
path = 'Img'
Dataset = LoadDataset.CustomDataSet("english.csv",path,62,LoadDataset.preprocessing_pipeline)

In [52]:
train_ENG,val_ENG,Test_ENG = LoadDataset.load_split(Dataset,64)

In [53]:
for batch in train_ENG:
    images, labels = batch
    print(images.shape)  # Print the shape of the batch of images
    print(labels.shape)  # Print the shape of the batch of labels
    break 

torch.Size([64, 1, 64, 64])
torch.Size([64, 62])
